<a href="https://colab.research.google.com/github/ShirishaReddyV/AI-Sandbox/blob/main/Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb sentence-transformers PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.3 MB/s eta 

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving small-business.pdf to small-business.pdf


In [ ]:
import fitz  # PyMuPDF
from langchain.schema import Document

def load_pdf_chunks(path, chunk_size=300):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()

    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    docs = [Document(page_content=chunk, metadata={"chunk_id": i}) for i, chunk in enumerate(chunks)]
    return docs

docs = load_pdf_chunks("small-business.pdf")  # update with your file path
print(f"✅ Loaded {len(docs)} chunks.")


✅ Loaded 569 chunks.


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./safety_rag_chroma")
vectordb.persist()


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./safety_rag_chroma")
vectordb.persist()

/tmp/ipython-input-10-1664471617.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-10-1664471617.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
def get_relevant_context(query, k=4):
    return vectordb.similarity_search(query, k=k)

query = "What are the emergency evacuation steps?"
relevant_docs = get_relevant_context(query)

context = "\n".join([doc.page_content for doc in relevant_docs])
print("📄 Retrieved Context:\n", context)


📄 Retrieved Context:
 as a step?
J Are all exits kept free of obstructions?
J Are at least two means of egress provided
from elevated platforms, pits, or rooms where
the absence of a second exit would increase
the risk of injury from hot, poisonous, corro-
sive, suffocating, flammable, or explosive sub-
stances?
J Are th
as a step?
J Are all exits kept free of obstructions?
J Are at least two means of egress provided
from elevated platforms, pits, or rooms where
the absence of a second exit would increase
the risk of injury from hot, poisonous, corro-
sive, suffocating, flammable, or explosive sub-
stances?
J Are th
ere sufficient exits to permit prompt
escape in case of emergency?
J Are special precautions taken to protect
employees during construction and repair
operations?
J Is the number of exits from each floor of a
building and the number of exits from the
building itself appropriate for the building
occu
ere sufficient exits to permit prompt
escape in case of emergency?
J Are sp

In [ ]:
from transformers import pipeline

qa_model = pipeline("text2text-generation", model="google/flan-t5-small", device="cpu")

prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {query}
"""

response = qa_model(prompt, max_new_tokens=150)
print("🤖 Answer:", response[0]['generated_text'].strip())


Device set to use cpu


🤖 Answer: J Are all exits kept free of obstructions? J Are at least two means of egress provided from elevated platforms, pits, or rooms where the absence of a second exit would increase the risk of injury from hot, poisonous, corro- sive, suffocating, flammable, or explosive sub- stances? J Are at least two means of egress provided from elevated platforms, pits, or rooms where the absence of a second exit would increase the risk of injury from hot, poisonous, corro- sive, suffocating, flammable, or explosive sub- stances? J Are at least two means of e
